# SCALA SPARK DRONES

This notebook allow spark operation for the drones project

## Packages 



In [2]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "org.apache.spark:spark-streaming_2.11:2.1.0,org.apache.spark:spark-streaming-kafka-0-8_2.10:2.1.0,com.google.code.gson:gson:2.4",
        "spark.jars.excludes": "org.scala-lang:scala-reflect,org.apache.spark:spark-tags_2.11"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1559225971971_0005,spark,idle,Link,Link,✔


SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1559225971971_0003,spark,idle,Link,Link,
2,application_1559225971971_0005,spark,idle,Link,Link,✔


## Kafka Topic



From Bash or other Unix shell:

```bash
CLUSTERNAME='spark-realscala'
USER='admin42
PASSWORD='RealScalaPass42Epita'
curl -u $USER:$PASSWORD -G "https://$CLUSTERNAME.azurehdinsight.net/api/v1/clusters/$CLUSTERNAME/services/ZOOKEEPER/components/ZOOKEEPER_SERVER" | jq -r '["\(.host_components[].HostRoles.host_name):2181"] | join(",")' | cut -d',' -f1,2
```

Replace the YOUR_ZOOKEEPER_HOSTS in the next cell with the returned value, and then run the cell

In [ ]:
%%bash
/usr/hdp/current/kafka-broker/bin/kafka-topics.sh --create --replication-factor 3 --partitions 8 --topic drones --zookeeper 'YOUR_ZOOKEEPER_HOSTS'

## Setup and Configuration

1. Link to drones data
2. kafka brokers configs

We need kafka brokers hosts, we can get them via bash:

```bash
CLUSTERNAME='spark-realscala'
USER='admin42
PASSWORD='RealScalaPass42Epita'
curl -u $USER:$PASSWORD -G "https://$CLUSTERNAME.azurehdinsight.net/api/v1/clusters/$CLUSTERNAME/services/KAFKA/components/KAFKA_BROKER" | jq -r '["\(.host_components[].HostRoles.host_name):9092"] | join(",")' | cut -d',' -f1,2
```


In [ ]:
import org.apache.spark.streaming._
import org.apache.spark.streaming.kafka._
import org.apache.kafka.clients.producer.{KafkaProducer, ProducerConfig, ProducerRecord}
import java.util.HashMap
import com.google.gson.Gson

// Vals for drones data TODO
// TODO
// TODO
// TODO


// Kafka configuration
// kafkaBrokers should contain a comma-delimited list of brokers. For example:
// kafkaBrokers = "wn0-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn1-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092,wn2-kafka.liftazhqudlunpo4tkvapo234g.dx.internal.cloudapp.net:9092"
val kafkaBrokers="your Kafka brokers"
val kafkaTopic="drones"

println("Finished configuring")

## Start Stream & Streaming

**TODO** : 
* Fix Stream ! MsgToJson, or already as json, etc...

In [4]:
// track the number of drones msgs emitted to Kafka
val numDronesMsg = sc.accumulator(0L, "Drones Msg sent to Kafka")

def createStreamingContext(): StreamingContext = {
    // Create the Kafka producer
    val producerProperties = new HashMap[String, Object]()
    producerProperties.put(ProducerConfig.BOOTSTRAP_SERVERS_CONFIG, kafkaBrokers)
    producerProperties.put(ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG,
                           "org.apache.kafka.common.serialization.StringSerializer")
    producerProperties.put(ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG,
                           "org.apache.kafka.common.serialization.StringSerializer")
    
    // set up the streaming context
    val ssc = new StreamingContext(sc, Seconds(5))
    // set up the stream, which we just convert to JSON
    // TODO !!!!
    val stream = ...
    // Write the data to Kafka
    stream.foreachRDD( rdd => {
        rdd.foreachPartition( partition => {
            val producer = new KafkaProducer[String, String](producerProperties)
            partition.foreach( record => {
                // Convert the data to JSON
                val gson = new Gson()
                val data = gson.toJson(record)
                val message = new ProducerRecord[String, String](kafkaTopic, null, data)
                
                producer.send(message)
                
                // Increment the counter
                numDronesMsg +=1
            })
            producer.close()
        })
    })
    ssc
}


accDronesMsg: org.apache.spark.Accumulator[Long] = 0

Writing to Kafka :

In [ ]:
val ssc = StreamingContext.getActiveOrCreate(createStreamingContext)
ssc.start()
// Timeout after 60 seconds
ssc.awaitTerminationOrTimeout(60000)
println("Finished writting " + numDronesMsg + " drones messages to Kafka")